In [1]:
import os
import cv2
import csv
import time
from ultralytics import YOLO
from utils import evaluate_patch_effectiveness_single

# --- CONFIG ---
image_folder = "/home/michele/hdd/stylegan3_error/adversarial_finetuning/src/overlay_results"
output_dir = "/home/michele/hdd/stylegan3_error/ablation_study"
os.makedirs(output_dir, exist_ok=True)

# Provide three YOLO model paths (replace with your actual checkpoints)
model_paths = [
    "yolov5n_best.pt",
    "yolov8n_best.pt",
    "yolov10s_best.pt",
]

# Target class id to evaluate (stop sign class)
target_class_id = 0

# Resize option: set to None to keep original, or a tuple (W, H)
resize_to = (640, 640)  # e.g. (640, 640) or None

# Log file (CSV) and text summary
csv_path = os.path.join(output_dir, "realimgtest_results.csv")
txt_path = os.path.join(output_dir, "realimgtest_log.txt")

# --- LOAD MODELS ---
models = []
for mp in model_paths:
    try:
        models.append((os.path.basename(mp), YOLO(mp)))
    except Exception as e:
        print(f"Failed to load model {mp}: {e}")

In [2]:
# --- EVALUATE ---
fieldnames = ["image", "model", "confidence", "notes"]
with open(csv_path, "w", newline="") as csvfile, open(txt_path, "w") as txtfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter='\t')
    writer.writeheader()

    image_files = sorted([f for f in os.listdir(image_folder) if f.lower().endswith((".png", ".jpg", ".jpeg"))])
    for img_name in image_files:
        img_path = os.path.join(image_folder, img_name)
        img = cv2.imread(img_path)
        if img is None:
            note = "failed_to_read"
            row_base = {"image": img_name, "model": None, "confidence": None, "notes": note}
            writer.writerow(row_base)
            txtfile.write(f"{row_base}\n")
            continue

        orig_h, orig_w = img.shape[:2]
        print(f"Image: {img_name} original size: ({orig_w}x{orig_h})")

        # optionally resize after printing original size
        proc_img = img
        if resize_to is not None:
            proc_img = cv2.resize(img, (resize_to[0], resize_to[1]))
            print(f" -> resized to: {resize_to}")

        # evaluate with each model
        for model_name, model in models:
            try:
                conf = evaluate_patch_effectiveness_single(yolov8_model=model, patched_image_np=proc_img, target_class_id=target_class_id)
                notes = ""
            except Exception as e:
                conf = None
                notes = f"error:{e}"

            row = {
                "image": img_name,
                "model": model_name,
                "confidence": conf,
                "notes": notes
            }
            writer.writerow(row)
            txtfile.write(f"{row}\n")
            print(f"  Model: {model_name} confidence: {conf} notes: {notes}")

print(f"Results saved to {csv_path} and {txt_path}")

Image: overlay_0_171539608576494.jpg original size: (2048x1536)
 -> resized to: (640, 640)


  Model: yolov5n_best.pt confidence: 0.8941667079925537 notes: 
  Model: yolov8n_best.pt confidence: 0.8910102844238281 notes: 
  Model: yolov10s_best.pt confidence: 0.7306398749351501 notes: 
Image: overlay_0_182573657075869.jpg original size: (2048x1536)
 -> resized to: (640, 640)
  Model: yolov5n_best.pt confidence: 0.8044121861457825 notes: 
  Model: yolov8n_best.pt confidence: 0.9007980823516846 notes: 
  Model: yolov10s_best.pt confidence: 0.887421190738678 notes: 
Image: overlay_0_237024841513641.jpg original size: (1024x576)
 -> resized to: (640, 640)
  Model: yolov5n_best.pt confidence: 0.4439207911491394 notes: 
  Model: yolov8n_best.pt confidence: 0.0 notes: 
  Model: yolov10s_best.pt confidence: 0.0 notes: 
Image: overlay_0_302085094922377.jpg original size: (2048x1152)
 -> resized to: (640, 640)
  Model: yolov5n_best.pt confidence: 0.7263836860656738 notes: 
  Model: yolov8n_best.pt confidence: 0.7700185179710388 notes: 
  Model: yolov10s_best.pt confidence: 0.0 notes: 
Im

In [ ]:
import csv
from collections import defaultdict

csv_path = "/home/michele/hdd/stylegan3_error/ablation_study/realimgtest_results.csv"
out_txt = "/home/michele/hdd/stylegan3_error/ablation_study/images_all_models_gt_0.7.txt"
threshold = 0.7

# try tab delimiter first, then comma
def load_rows(path):
    with open(path, newline='') as f:
        reader = csv.DictReader(f, delimiter='\t')
        if reader.fieldnames and 'confidence' in reader.fieldnames:
            return list(reader)
    # fallback to comma
    with open(path, newline='') as f:
        reader = csv.DictReader(f, delimiter=',')
        return list(reader)

rows = load_rows(csv_path)

# accumulate models that pass threshold per image
passing = defaultdict(set)
for r in rows:
    img = r.get('image') or r.get('Image') or r.get('filename')
    model = r.get('model') or r.get('Model')
    conf_s = r.get('confidence') or r.get('Confidence') or ""
    try:
        conf = float(conf_s)
    except Exception:
        continue
    if conf > threshold and img and model:
        passing[img].add(model)

# require images that pass for at least 3 different models (all three)
result_images = [img for img, models in passing.items() if len(models) >= 3]

# save to txt
with open(out_txt, 'w') as f:
    for img in sorted(result_images):
        f.write(f"{img}\n")

print(f"Found {len(result_images)} images where >=3 models have confidence > {threshold}.")
print(f"Wrote list to: {out_txt}")

Found 151 images where >=3 models have confidence > 0.7.
Wrote list to: /home/michele/hdd/stylegan3_error/ablation_study/images_all_models_gt_0.7.txt
